In [10]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install itertools

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from tqdm import tqdm
import random
import os
from datetime import datetime
import numpy as np
import torch
import torch.nn as nn
import itertools
# from google.colab import drive
# drive.mount('/gdrive')

In [13]:
# finance-datareader(주식 데이터)/pandas_market_calendars(주말, 공휴일 제거용) 다운로드
!pip install -U finance-datareader
!pip install pandas_market_calendars


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
pip install -U finance-datareader

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import FinanceDataReader as fdr
import pandas_market_calendars as mcal
import random

# 한국거래소 전체 상장종목
df_krx = fdr.StockListing('KRX')
df_krx.head()

In [ ]:
df_krx["Name"]

사용자로부터 종목 이름, 날짜 입력받아 각각 일별 등락률 가져오기

In [ ]:
# 사용자로부터 종목 이름 입력 받기
selected_companies_names = []
for i in range(3):
    company_name = input(f"종목 이름을 입력하세요 {i+1}: ")
    selected_companies_names.append(company_name)

# 사용자로부터 날짜 입력 받기
input_date = input("형식에 맞춰 날짜를 입력하세요. (YYYY-MM-DD): ")
start_date = pd.to_datetime(input_date) - pd.DateOffset(years=1)

# 종목 이름을 종목 코드로 변환하고 등락률을 저장할 데이터프레임 초기화
df_returns = pd.DataFrame()
df_close_price = pd.DataFrame()


# 각 종목에 대해 데이터 불러오고 등락률 가져오기
for company_name in selected_companies_names:
    # 종목 코드 가져오기
    try:
        company_code = df_krx[df_krx['Name'] == company_name]['Code'].values[0]
        df_stock = fdr.DataReader(company_code, start_date, input_date)
        df_returns[company_name] = df_stock['Change']
        df_close_price[company_name] = df_stock['Close']
    except IndexError:
        print(f"입력하신 {company_name}은 존재하지 않습니다.")

df_returns

위에서 가져온 일별 등락률을 바탕으로 <br>
개별 주식 평균 등락률 E(r1), E(r2), E(r3) 구하기<br>
E(r) : 개별 주식의 일별 등락률 평균 x 날짜(일 수)

In [ ]:
daily_stock_average_d = []
average_returns = df_returns.mean()
average_returns = average_returns * len(df_returns)
for value in average_returns:
    daily_stock_average_d.append(value)

print(average_returns)
print("daily_stock_average_d:", daily_stock_average_d)

개별 주식 변동성 σ1, σ2, σ3 구하기<br>
(개별일별등락률의 모집단표준편차) * (전체 일 수의 제곱근)

In [ ]:
personal_stock_v = []
std_dev = df_returns.std()
total_days = len(df_returns)
volatility = std_dev * (total_days ** 0.5)
for value in volatility:
    personal_stock_v.append(value)

print(volatility)
print("personal_stock_v:", personal_stock_v)

3종목간의 상관계수 p12, p13, p23 구하기<br>
3 종목의 1년치 종가를 가져와서 3 종목간의 각각의 상관계수 구하기

In [ ]:
print(selected_companies_names)
df_close_price

In [ ]:
# 각 종목 간의 상관계수 계산
p12_correlation = df_close_price[selected_companies_names[0]].corr(df_close_price[selected_companies_names[1]])
p13_correlation = df_close_price[selected_companies_names[0]].corr(df_close_price[selected_companies_names[2]])
p23_correlation = df_close_price[selected_companies_names[1]].corr(df_close_price[selected_companies_names[2]])

# 결과 출력
print(f"1,2번 종목의 상관계수: {p12_correlation}")
print(f"1,3번 종목의 상관계수: {p13_correlation}")
print(f"2,3번 종목의 상관계수: {p23_correlation}")

In [ ]:
stock_corr = []
stock_corr.append(p12_correlation)
stock_corr.append(p13_correlation)
stock_corr.append(p23_correlation)
stock_corr

원래 총 조합 5151개 중 최대 0.7, 최소 0.1 조건을 만족하는 조합만 추출

In [ ]:
def find_combinations():
    possible_values = [i / 100 for i in range(101)] # 0부터 1까지 0.01 간격으로 조합 추출
    combinations = itertools.product(possible_values, repeat=3)

    valid_combinations = []
    valid_indices = []

    for i, combo in enumerate(combinations):
        if round(sum(combo), 2) == 1.0 and all(0.1 <= x <= 0.7 for x in combo): # 최대 0.7, 최소 0.1은 가지는 조합만 추출
            valid_combinations.append(combo)
            valid_indices.append(i)

    return valid_combinations, valid_indices

result, valid_indices = find_combinations()

print("조합의 수:", len(result))
print("일부 예시 조합:", result[:10])

포트수익률 E(rt) 구하기<br>
(x1 x E(r1) + x2 x E(r2) + x3 x E(r3))

In [ ]:
port_returns = []

for combo in result:
    port_return = sum(x * er for x, er in zip(combo, daily_stock_average_d))
    port_returns.append(port_return)

print("포트수익률 개수:", len(port_returns))
print(port_returns[:10])

In [ ]:
pip install pykrx

In [ ]:
from pykrx import bond

# 사용자에게 날짜를 입력받음
date = input("날짜를 'YYYYMMDD' 형식으로 입력하세요: ")

# 입력받은 날짜로 데이터프레임 생성
df = bond.get_otc_treasury_yields(date)

# 국고채 3년의 수익률 데이터만 추출하여 출력
kukko_3year_yield = df.loc['국고채 3년', '수익률']
print(kukko_3year_yield)
#from pykrx import bond

# 사용자에게 날짜를 입력받음
#date = input("날짜를 'YYYYMMDD' 형식으로 입력하세요: ")

# 입력받은 날짜로 데이터프레임 생성
#df = bond.get_otc_treasury_yields(date)

# 데이터프레임 출력
#print(df)
#goverment_bond = input(df)

사용자에게 날짜에 맞는 3년 국채를 입력받기



σ(t) 값 구하기<br>
(x1 ^ 2 x σ1 ^ 2) + (x2 ^ 2 x σ2 ^ 2) + (x3 ^ 2 x σ3 ^ 2) + (2x1x2σ1σ2p12) + (2x2x3σ2σ3p23) + (2x1x3σ1σ3p13)

In [ ]:
port_vol = []

for combo in result:
    term1 = sum(x ** 2 * sigma ** 2 for x, sigma in zip(combo, personal_stock_v))
    term2 = 0

    for i in range(len(combo) - 1):
        for j in range(i + 1, len(combo)):
            idx = i * (len(combo) - 1) + j - 1
            corr_idx = min(idx, 2)
            term2 += 2 * combo[i] * combo[j] * personal_stock_v[i] * personal_stock_v[j] * stock_corr[corr_idx]

    port_volatility = (term1 + term2) ** 0.5
    port_vol.append(port_volatility)

print("포트폴리오 변동성의 개수:", len(port_vol))
print(port_vol[:10])

Cal 기울기 구하기<br>
(E(rt)-Rf)/σ(t)<br>
위에서 조건을 만족하는 (최대 0.7, 최소 0.1은 배정) 조합 중에 최대 Cal 기울기 값을 가져오도록

In [30]:
# government_bond = Rf
Rf = float(government_bond)
Cal_values = []

# port_returns_numeric = [float(val) for val in port_returns]

for i in range(len(port_returns)):
    Cal = (port_returns[i] - Rf) / port_vol[i]
    Cal_values.append(Cal)

print("Cal 값의 개수:", len(Cal_values))
print(Cal_values[:10])

Cal 값의 개수: 2391
[-1.0706423583530567, -1.0707621908866871, -1.0708791075321105, -1.0709928343001895, -1.0711030775901522, -1.071209522712411, -1.0713118322889594, -1.0714096445201402, -1.071502571305445, -1.0715901962047527]


Max Cal 기울기<br>
최대 기울기와 조합을 찾음

In [31]:
max_cal_value = max(Cal_values)
max_cal_index = Cal_values.index(max_cal_value)
max_result = result[max_cal_index]

print("Cal 기울기 최대값 : ", max_cal_value)
print("Max Cal 위치 : ", max_cal_index)
print("Max Cal 조합 : ", max_result)

Cal 기울기 최대값 :  -0.9557204734641668
Max Cal 위치 :  2390
Max Cal 조합 :  (0.7, 0.2, 0.1)


위험비율, 투자금액 입력받기
->부동소수점 문제 때문에 int로 받고 반올림하는 식으로

In [32]:
Investment = int(input(f"투자하실 금액을 입력하세요: "))
print("투자비율을 입력하세요.")
no_danger = float(input(f"무위험 투자비율: "))
high_danger = float(input(f"고위험 투자비율: "))

invest_amount = []
real_invest = (Investment * (high_danger * 0.1))
rf = Investment - real_invest
for i in range(len(max_result)):
  price = real_invest * max_result[i]
  invest_amount.append(price)

print("-------------------------------------")
print("총 투자 금액 : {:,.0f}".format(Investment))
print("rf : {:,.0f}".format(rf))
for i in range(len(selected_companies_names)):
  print("{} 투자 금액: {:,.0f}".format(selected_companies_names[i], invest_amount[i]))

투자하실 금액을 입력하세요: 1000000000
투자비율을 입력하세요.
무위험 투자비율: 7
고위험 투자비율: 3
-------------------------------------
총 투자 금액 : 1,000,000,000
rf : 700,000,000
삼성전자 투자 금액: 210,000,000
SK하이닉스 투자 금액: 60,000,000
베른 투자 금액: 30,000,000


In [ ]:
Investment = int(sys.argv[4])
danger = int(sys.argv[5])
#고위험 = sys.argv[5] = 7, 중위험 = sys.argv[5] = 5, 저위험 = sys.argv[5] = 3

invest_amount = []
real_invest = (Investment * (danger * 0.1))
rf = Investment - real_invest
for i in range(len(max_result)):
  price = real_invest * max_result[i]
  invest_amount.append(price)
